In [22]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
import secrets
import os


In [23]:
def encrypt_aes(key: bytes, iv: bytes, plaintext: bytes) -> bytes:
    cipher = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    )
    encryptor = cipher.encryptor()
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    return ciphertext

In [24]:
def gerar_metadata(arquivo_original: str, chave: bytes, arquivo_metadata: str = None) -> str:
    with open(arquivo_original, "rb") as f:
        conteudo = f.read()

    iv = secrets.token_bytes(16)

    ciphertext = encrypt_aes(chave, iv, conteudo)
    fingerprint = ciphertext[-16:]  # Últimos 16 bytes

    header = bytearray()
    header += b'CF'                     # Identificador (2 bytes)
    header += bytes([0x01])             # Versão (1 byte)
    header += bytes([0x01])             # Algoritmo AES (1 byte)
    header += bytes([0x01])             # Modo CBC (1 byte)
    header += iv                        # IV (16 bytes)
    header += fingerprint               # Fingerprint (16 bytes)
    header += bytes(11)                 # Reservado (11 bytes)

    if arquivo_metadata is None:
        base_name = os.path.basename(arquivo_original)
        arquivo_metadata = os.path.splitext(base_name)[0] + ".meta"

    with open(arquivo_metadata, "wb") as f:
        f.write(header)

    return arquivo_metadata

In [25]:
def verificar_integridade(arquivo_original: str, arquivo_metadata: str, chave: bytes) -> bool:
    """
    :param arquivo_original: Caminho do arquivo original
    :param arquivo_metadata: Caminho do arquivo de metadados
    :param chave: Chave AES usada na >>>geração<<<
    """

    with open(arquivo_metadata, "rb") as f:
        metadata = f.read(48)

    ident = metadata[0:2]
    version = metadata[2]
    algo = metadata[3]
    mode = metadata[4]
    iv = metadata[5:21]
    fingerprint_original = metadata[21:37]
    reserved = metadata[37:48]

    if ident != b'CF' or version != 0x01 or algo != 0x01 or mode != 0x01:
        raise ValueError("Arquivo de metadados inválido ou corrompido")

    with open(arquivo_original, "rb") as f:
        conteudo = f.read()

    ciphertext = encrypt_aes(chave, iv, conteudo)
    fingerprint_atual = ciphertext[-16:]


    return fingerprint_original == fingerprint_atual

In [28]:
nome_arquivo = "arquivo_teste.pdf"
chave = bytes([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4])  # 16 bytes


arquivo_meta = gerar_metadata(nome_arquivo, chave)
print(f"Arquivo de metadados gerado: {arquivo_meta}")


integro = verificar_integridade(nome_arquivo, arquivo_meta, chave)
print(f"Arquivo íntegro? {integro}")


with open(nome_arquivo, "ab") as f:
    f.write(b" MODIFICADO")

integro = verificar_integridade(nome_arquivo, arquivo_meta, chave)
print(f"Arquivo íntegro após modificação? {integro}")

Arquivo de metadados gerado: arquivo_teste.meta
Arquivo íntegro? True
Arquivo íntegro após modificação? False
